<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Агрегация, ограниченная диффузией

### Examples - Chemistry
<section class="post-meta">
By Jonas Tjemsland, Andreas S. Krogen and Jon Andreas Støvneng
</section>
Last edited: March 22nd 2018 
___

## Вступление

Возможно, первое серьезное исследование агрегации с ограниченной диффузией было проведено Т. А. Виттеном и Л. М. Сандером и опубликовано ими в 1981 году. Как следует из названия, модель описывает случайную агрегацию, формирование ряда объектов в кластер. Эта модель, среди прочего, используется для описания диффузии и агрегации ионов меди в электролитическом растворе на электродах, как показано на рисунке 1.

<img src="images/DLA_Cluster.JPG" width="400">  
**Рисунок 1:** Кластер агрегации с ограниченной диффузией (DLA). Медный агрегат, образующийся в электроосаждающей ячейке. (Источник: Кевин Джонсон, [wikipedia](https://commons.wikimedia.org/wiki/File:DLA_Cluster.JPG))

Виттен и Сандер описывают свою модель следующим образом [1]:  
*Наша модель является вариантом модели Эдема, начальным состоянием которой является начальная частица в начале решетки. Вторая частица добавляется в каком-то случайном месте на большом расстоянии от источника. Эта частица движется случайным образом, пока не попадет на участок, прилегающий к семени. Затем движущаяся частица становится частью кластера. Другая частица теперь вводится в случайную удаленную точку, и она движется случайным образом, пока не присоединится к кластеру, и так далее. Если частица касается границ решетки во время своего случайного блуждания, она удаляется и вводится другая.*

У Пола Бурка [2] есть более красочное описание, включающее городскую площадь, окруженную тавернами:  
*Пьяницы покидают таверны и беспорядочно шатаются по площади, пока, наконец, не спотыкаются об одного из своих бесчувственных товарищей, после чего, убаюканные звуками мирного храпа, ложатся и засыпают. Структура, похожая на усик, представляет собой вид с высоты птичьего полета на спящую толпу утром.*



Корреляции плотности в модельных агрегатах уменьшаются с расстоянием с дробным степенным законом, а радиус агрегации имеет степенное поведение [1], которое совпадает с экспериментами (как показано на рисунке 1). 

Как всегда, мы начинаем с включения библиотек и установки общих параметров рисунка.

In [ ]:
%matplotlib inline
import numpy as np
from math import ceil, floor
import matplotlib.pyplot as plt
import progressbar
import warnings
warnings.filterwarnings('ignore')
from scipy.optimize import curve_fit

In [ ]:
# Set some figure parameters
newparams = {'savefig.dpi': 200, 'figure.figsize': (20, 10), 
             'font.size': 25, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

## Моделирование

Мы будем рассматривать квадратную решетку, где каждая позиция может содержать или не содержать частицу. Случайное блуждание частиц может быть описано как [броуновское движение](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/brownian_motion_intro.ipynb): временная эволюция положения частицы принимает форму дискретных шагов, и на каждом шаге существует равная вероятность того, что частица переместится на расстояние $h$ в каждом направлении. В двух измерениях вероятность того, что частица движется *вверх*, *вниз*, *влево* и *вправо*, имеет вероятность $1/4$.

Предполагается, что плотность настолько мала, что взаимодействие между двумя (случайно движущимися) частицами является незначительным. Таким образом, мы можем рассматривать по одной частице за раз. Частица становится частью кластера, если ее положение на квадратной решетке смежно с некоторой частью кластера. В этом блокноте мы определим *смежные* как *вертикальные, диагональные или горизонтальные соседние точки*. То есть каждая точка на решетке имеет восемь соседних точек. Обратите внимание, что Виттен и Сандер использовали в своей оригинальной статье только горизонтальные и вертикальные соседние точки. Кроме того, мы также введем так называемую *липкость*, которая представляет собой вероятность того, что частица прилипнет к кластеру в каждой соседней точке.

Обратите внимание, что теоретически частица может удалиться, и поэтому мы должны *что-то сделать* с частицами, которые *находятся вне диапазона* (либо попадают на край решетки, либо находятся за пределами указанного диапазона). Есть несколько способов сделать это. Можно, например, реализовать периодические граничные условия (частица, которая попадает в правую часть решетки или в определенный диапазон, Появляется слева и т.д.), Игнорировать шаги, которые перемещают частицы за пределы диапазона, Или игнорировать частицу и вводить новую. Они будут иметь примерно одинаковые визуальные результаты [2]. Мы будем реализовывать последнее.

В исходной модели частицы, как говорят, вводятся в *удаленную точку*. Чтобы сократить вычислительное время, мы будем вводить частицы, случайным образом распределенные по окружности, радиус которой немного больше максимального радиуса кластера.

Последнее замечание: языки сценариев, такие как Python, довольно медленные по сравнению с компилируемыми языками, такими как Fortran и C/C++. Многие функции в NumPy и SciPy вызывают скомпилированные модули (см., например, наше руководство по [F2PY](https://www.numfys.net/howto/F2PY/)). Модель DLA очень требовательна к вычислениям, и поэтому мы хотим использовать эти скомпилированные модули. В приведенной ниже функции мы используем функции NumPy для вычисления последовательности из 200 шагов за один раз, что значительно сокращает вычислительное время, но код может быть немного сложным. В случае, когда липкость меньше единицы, мы продолжаем вычислять отдельные шаги.

In [ ]:
def DLA(particles, seed, stickiness, L=1000, STEP_CHUNCK=200, range_length=100, start_length=10):
    """ Performs the Diffusion Limited Aggregation model on a lattice for a 
    given amount of particles and a given seed. Try to optimize optional arguments!
    Parameters:
        particles:    int. Number of particles.
        seed:         array-like, mxn, m,n < L. Seed.
        stickiness:   float. 'Sticking' probability. Should be in the range [0,1].
        L:            int. Size of the lattice. The lattice will become a (2L+1)x(2L+1).
        STEP_CHUNCK:  int. Specifies the number of steps calculated at once.
        range_length: int. Length at which the particles are considered "out of range".
        start_length: int. Length at which the particles are created.
    Output:
        lattice:     (2L+1)x(2L+1) numpy array. Array describing the cluster. The
                     entries are 0 at empty position and n for the n'th particle.
        max_radius:  float. The maximum radius of the lattice.
    """

    # Define possible steps and adjacent cells
    STEPS = np.array([(0,1), (0,-1), (1,0), (-1,0)])
    ADJACENT = np.array([(0,1), (0,-1), (1,0), (-1,0), (1,1), (1,-1), (-1,1), (-1,-1)])

    # Create empty lattice
    lattice_size = 2*L + 1
    lattice = np.zeros((lattice_size, lattice_size), dtype='int32')

    # Create a cluster. That is, a lattice holding all 'adjacent cells'
    cluster = (lattice != 0)
    # Set the seed in the midle of the lattice
    seed = (seed != 0) # Set all entries to one or zero
    seed_size = np.array([np.size(seed,0), np.size(seed,1)])
    l = seed_size/2
    cluster[ceil(L - l[0]):ceil(L + l[0]), ceil(L - l[1]):ceil(L + l[1])] = seed
    # We define the 'start maximal radius' as being the maximal radius of the seed.
    max_radius = (np.size(seed,0)**2 + np.size(seed,1)**2)**.5
    lattice = np.copy(cluster)*particles

    ### Performing the simulation ###
    angles = np.random.rand(particles)*2*np.pi # One random angle per particle

    bar = progressbar.ProgressBar()
    for i in bar(range(particles)):

        # Setting inital position
        x = int((max_radius + start_length)*np.cos(angles[i]) + L)
        y = int((max_radius + start_length)*np.sin(angles[i]) + L)

        while True:
            # Performing one step chunck og STEP_CHUNK number of steps. This idea was inspired by
            # https://github.com/hemmer/pyDLA/blob/numpyFast/dla.py
            steps = np.random.randint(0,4,STEP_CHUNCK)
            steps = np.cumsum(STEPS[steps], axis=0)
            steps[:,0] += x
            steps[:,1] += y 
            # Check at which positions in the chunck the particle is adjacent to the cluster
            adjacent = cluster[steps[:,0], steps[:,1]]

            if np.any(adjacent): # If at least one of the positions are adjacent
                # If the stickiness is 100%, the particle will be stuck at the first adjacent position
                first_hit = adjacent.nonzero()[0][0]
                x = steps[first_hit, 0]
                y = steps[first_hit, 1]
                current_radius = (x - L)**2 + (y - L)**2
                # If not, we should perform only single steps at the time, and only allow steps that
                # does not collide with existing particles. At each adjacent position, there are a
                # probability of sticking. If the particle goes outside the maximum radius of the
                # cluster, we forget about it and introduce a new
                if (stickiness < 1):
                    is_attached = (np.random.rand(1) < stickiness)
                    inside = (current_radius < (max_radius + 2)**2)
                    while ((not is_attached) and inside):
                        while True:
                            r = np.random.randint(1,5)
                            if (r == 1 and (not lattice[x + 1, y])):
                                x += 1
                                break
                            elif (r == 2 and (not lattice[x - 1, y])):
                                x -= 1
                                break
                            elif (r == 3 and (not lattice[x, y + 1])):
                                y += 1
                                break
                            elif (r == 4 and (not lattice[x, y - 1])):
                                y -= 1
                                break
                        if (cluster[x, y]):
                            is_attached = (np.random.rand(1) < stickiness)
                        current_radius = (x - L)**2 + (y - L)**2
                        inside = (current_radius < (max_radius + 2)**2)
                    if (not inside):
                        new_angle = np.random.rand(1)*2*np.pi
                        x = int((max_radius + start_length)*np.cos(new_angle) + L)
                        y = int((max_radius + start_length)*np.sin(new_angle) + L)
                    else:
                        lattice[x, y] = i
                        cluster[ADJACENT[:, 0] + x, ADJACENT[:, 1] + y] = True
                        if (current_radius > max_radius**2):
                            max_radius = current_radius**.5
                        break
                else:
                    # Put the particle number into the lattice at current position,
                    # if it is adjacent to the cluster
                    lattice[x, y] = i
                    # Update adjacent cells
                    cluster[ADJACENT[:, 0] + x, ADJACENT[:, 1] + y] = True
                    # Update the cluster radius
                    if (current_radius > max_radius**2):
                        max_radius = current_radius**.5
                    break

            else:
                x = steps[-1,0]
                y = steps[-1,1]
                # If the last posision in the chunck is out of range, then introduce a new particle
                current_radius = (x - L)**2 + (y - L)**2
                if (current_radius > (max_radius + range_length)**2):
                    new_angle = np.random.rand(1)*2*np.pi
                    x = int((max_radius + start_length)*np.cos(new_angle) + L)
                    y = int((max_radius + start_length)*np.sin(new_angle) + L)
    return lattice, max_radius

Давайте начнем с выполнения моделирования, скажем, с 10000 частиц и одной точки в качестве затравки для различной липкости. Мы также хотим обрезать область (помните, что решетка должна быть намного больше радиуса кластера) и построить график результатов.

In [ ]:
def DLA_sim(particles=10000, stickiness=1, seed=np.ones((1,1))):
    """ Calls DLA() and visualizes the results.
    Parameters: Same as DLA()
    Output:
        lattice_gray_cut: numpy array. Cropped array describing the cluster. The
                          entries are 0 at empty position and n for the n'th particle.
        lattice_cut:      numpy array. Cropped array describing the cluster. The
                          entries are 0 at empty position and 1 otherwise.
        max_radius:       float. The maximum radius of the lattice.
    """
    # Perform simulation
    lattice, max_radius = DLA(particles, seed, stickiness)
    # Create a corresponding black/white lattice
    lattice_gray = (lattice != 0)
    # Crop
    mask = np.ix_(lattice_gray.any(1),lattice_gray.any(0)) # Create a 'crop mask'
    lattice_gray_cut = lattice_gray[mask]
    lattice_cut = lattice[mask]
    print("Maximum radius: ", max_radius)
    # Plot results
    plt.figure()
    plt.suptitle("DLA-cluster with %d particles and stickiness %.2f" % (particles, stickiness))
    plt.subplot(121)
    plt.imshow(lattice_cut, interpolation='none')
    plt.subplot(122)
    plt.imshow(lattice_gray_cut, interpolation='none', cmap='gray')
    plt.show()
    return lattice_gray_cut, lattice_cut, max_radius

In [ ]:
lattice = DLA_sim(stickiness=1.00)
DLA_sim(stickiness=0.50);
DLA_sim(stickiness=0.10);

На левом изображении хорошо видно, что новые частицы часто прилипают к кластеру на периферии. При меньшей вероятности прилипания радиус уменьшается. То есть структура становится более компактной, а "щупальца" более "ветвистыми". Как мы увидим, меньшая липкость дает меньшую фрактальную размерность $D$.

Это очень похоже на рисунок 1, и результат в статье!

## Фракталы
Во многих отношениях эти кластеры напоминают так называемые фракталы, математический набор, который демонстрирует повторяющийся шаблон, самоподобный в различных масштабах [3]. Один пример фрактала показан на рисунке 2 ниже. Этот фрактал на самом деле создается с использованием довольно простого метода, называемого *итерационными функциональными системами (IFSs)*, с несколькими входными параметрами [4]. Изменяя эти параметры, можно создать множество известных и художественных фракталов. 

Глядя на рисунок, можете ли вы убедить себя в том, что фракталы встречаются в природе? Некоторые явления, известные своими фрактальными особенностями, включают речные сети, горные хребты, молнии, деревья, рога горных козлов, водоросли, геометрическую оптику, узоры окраски животных, кристаллы и ДНК! [3]

![Barsley Fern fractal](https://www.numfys.net/media/notebooks/images/barnsley-fern-fractal.png)
**Рисунок 2** Фрактал папоротника Барсли, созданный с использованием Python [5]

Обратите внимание, как одна ветвь на рисунке 2 разветвляется на множество более мелких ветвей. Каждый из них, в свою очередь, напоминает *родительскую ветвь*. Это также относится и к нашим кластерам DLA!

## Фрактальная размерность

Фракталы имеют конечную площадь, но бесконечный периметр. Рассмотрим наши кластеры DLA: используя большую измерительную палочку, мы бы не дотянулись между ветвями, но с помощью маленькой измерительной палочки мы бы это сделали. Таким образом, фракталы не могут быть описаны с помощью обычных евклидовых измерений, они в общем случае имеют нецелочисленную фрактальную размерность $D$. Это говорит нам кое-что о том, насколько плотен фрактал. Теоретически фрактальная размерность кластера DLA составляет $D \approx 1.71$. Однако это верно только в том случае, если частицы не ограничены решеткой.

Предположим, что $d$-мерный фрактал покрыт набором $d$-мерных фигур (например, сфер или коробок в двух измерениях) линейного размера $\epsilon$ (квадрат $L\times L$ будет иметь $\epsilon\propto L$, и мы можем выбрать $\epsilon = L$. Для сферы мы можем, например, выбрать $\epsilon$ в качестве радиуса или диаметра сферы или даже окружности! Важно именно масштабирование).  Затем количество таких фигур будет масштабироваться как
$$N \sim \epsilon^{-d}$$
[6]. В евклидовом пространстве $D$ всегда является целым числом. Это уравнение можно записать в виде
$$D \equiv -\frac{\log N(\epsilon)}{\log \epsilon}.$$

Существует несколько способов оценить фрактальную размерность нашего дискретного кластера DLA. Виттен и Сандер [1] использовали корреляционную функцию плотности для анализа плотности. Они обнаружили, что $\rho = -0.34$, и, в свою очередь, может быть показано, что это соответствует фрактальной размерности $D=1.66$. В этом блокноте мы скорее будем использовать более интуитивно понятный алгоритм подсчета квадратов. Обратите внимание, что это не точная наука: мы всегда будем ожидать, что $D$ меньше, чем теоретические $1.71$, а может быть, даже ниже, чем $1.66$!

## Алгоритм подсчета квадратов
Теперь мы приблизим фрактальную размерность, используя алгоритм подсчета квадратов. Идея довольно проста: мы выбираем коробку размером $\epsilon$, покрываем ее сеткой и подсчитываем, сколько из этих ячеек содержит хотя бы одну частицу. Затем мы проверяем, как количество ячеек $N$ масштабируется с размером $\epsilon$. 
$$D \equiv \lim_{\epsilon \to 0}\frac{\log N(\epsilon)}{\log (1/\epsilon)}.$$

In [ ]:
def box(n, lattice):
    """ Divide lattice into boxes and count how many that has
    a positive sum of entries. The boxes are Lx/n x Ly/n, where
    the lattice is an (Lx, Ly)-array.
    Parameter:
        n:         int. Division fraction.
        lattice:   array-like, 2D. The cluster being analyzed. Should be 
                   cropped to maximize performance.
    Returns:
        num:       int. Number of boxes with a positive sum of entries.
    """
    L = [np.size(lattice,0), np.size(lattice,1)]
    Nx = int(L[0]/n)
    Ny = int(L[1]/n)
    num = 0
    for i in range(Nx):
        for j in range(Ny):
            contained = np.sum(lattice[int(i*n):int(i*n+n),int(j*n):int(j*n+n)])
            if (contained):
                num += 1
    for i in range(Nx):
        contained = np.sum(lattice[int(i*n):int(i*n+n), int(Ny*n):int(L[1]+1)])
        if (contained):
            num += 1
    for i in range(Ny):
        contained = np.sum(lattice[int(Nx*n):int(L[1]+1), int(i*n):int(i*n+n)])
        if (contained):
            num += 1
    contained = np.sum(lattice[int(Nx*n):int(L[0]+1), int(Ny*n):int(L[1]+1)])
    if (contained):
        num += 1
    return num

def paket(lattice, start_potent=1, end_potent=7, num_points=20):
    """ Calls box() repeatedly with different box sizes, exponentially distibuted.
    Parameters:
        lattice:      array-like, 2D. The cluster being analyzed. Should be 
                      cropped to maximize performance.
        start_potent: int. Size of the first box as a fraction of the lattice.
        end_potent:   int. Size of the last box as a fraction of the lattice.
        num_points:   int. Number of calls to box() with n logarithmically
                      distibuted between start_potent and end_potent.
    Return:
        X:            1xnum_points numpy array. Linear box size.
        Y:            1xnum_points array. Number of boxes that contains at least a particle.
    """
    L = np.size(lattice,0)
    X = np.logspace(start_potent, end_potent, num=num_points, endpoint=True, base=2)
    Y = []
    for i in range(len(X)):
        Y.append(box(X[i],lattice))
    return X, Y

Теперь мы просто вызываем описанные выше функции для одного из наших кластеров, выполняем регрессию и строим график результата!

In [ ]:
X, Y = paket(lattice[0], 1, 7, 20)
x = np.log(X)
y = np.log(Y)
coeff = np.polyfit(x, y, 1)
print("Fractal dimension: D = %.2f" % -coeff[0])

plt.figure()
plt.plot(x,y,'o')
xi = np.array([min(x), max(x)])
plt.plot(xi, coeff[0]*xi + coeff[1])
plt.xlabel("$\log(\epsilon)$")
plt.ylabel("$\log(N(\epsilon))$")
plt.show()

## Поиграйте с разными семенами!

In [ ]:
particles = 5000
seed = np.ones((100,100))
DLA_sim(particles=particles, seed=seed);

In [ ]:
seed = np.zeros((100,20))
seed[:,5] = 1
seed[:,-5] = 1
DLA_sim(particles=5000, seed=seed);

### References

[1] [T. A. Witten Jr, L. M. Sander, Phys. Rev. Lett. 47, 1400 (1981)](http://journals.aps.org/prl/abstract/10.1103/PhysRevLett.47.1400)  
[2] P. Bourke: Diffusion Limited Aggregation, http://paulbourke.net/fractals/dla/, June 1991 [acquired: 01.10.2017]  
[3] Wikipedia: Fractal, https://en.wikipedia.org/wiki/Fractal [acquired: 01.26.2017]  
[4] Wikipedia: Iterated Funtion System (IFS), https://en.wikipedia.org/wiki/Iterated_function_system [acquired: 01.26.2017]  
[5] ActiveState code, by user FB26, http://code.activestate.com/recipes/578003-ifs-fractals-using-iteration-method/, 01.07.2012 [acquired: 01.26.2017]   
[6] Wikipedia: Fractal Dimension, https://en.wikipedia.org/wiki/Fractal_dimension [acquired: 01.26.2017]  